In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [2]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-18 11:36:00,113 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-18 11:36:00,114 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-18 11:36:02,951 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:ca6ccd63f270d603ffa9bdf5a7ced184d6339796
2017-06-18 11:36:03,340 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-18 11:36:06,184 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-18 11:36:06,185 [INFO] [holmes.experiment.staging] Commit staged
2017-06-18 11:36:06,186 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-18 11:36:06,263 [INFO] [holmes.experiment.staging] Extracting default config from package


In [3]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [1,2]
hungarian_thresholds = [15,20]
minimum_detections = [10,15]

In [4]:
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

32 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M00-ChvUtd-000-01-15-10.json


In [5]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-1-9", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-1-9",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-18 11:36:16,536 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9


In [6]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [7]:
experiment.stage_experiment()

2017-06-18 11:36:21,842 [INFO] [holmes.experiment.base] staging measurement M00-ChvUtd-000-01-15-10
2017-06-18 11:36:22,309 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M00-ChvUtd-000-01-15-10.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M00-ChvUtd-000-01-15-10.json
2017-06-18 11:36:22,703 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz
2017-06-18 11:36:24,012 [INFO] [holmes.experiment.base] staging measurement M01-PS0304-001-01-15-10
2017-06-18 11:36:24,460 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M01-PS0304-001-01-15-10.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps

2017-06-18 11:36:43,558 [INFO] [holmes.experiment.base] staging measurement M22-PP0214-000-02-15-15
2017-06-18 11:36:44,016 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M22-PP0214-000-02-15-15.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M22-PP0214-000-02-15-15.json
2017-06-18 11:36:44,400 [INFO] [holmes.experiment.base] staging measurement M23-ER0429-001-02-15-15
2017-06-18 11:36:44,905 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M23-ER0429-001-02-15-15.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M23-ER0429-001-02-15-15.json
2017-06-18 11:36:45,291 [INFO] [holmes.experiment.base] staging measurement M24-ChvUtd-000-02-20-10
2017-06-18 11:36:45,737 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp7dhya0yo/M24-ChvUtd-000-02-20-10.json, dest

{'experiment_id': 'PTA-95-LT-HT-DET-1-9',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M00-ChvUtd-000-01-15-10.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-9',
   'job_id': 'PTA-95-LT-HT-DET-1-9-M00-ChvUtd-000-01-15-10',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-ChvUtd-000-01-15-10',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz'},
  {'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M01-PS0304-001-01-15-10.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-9',
   'job_id': 'PTA-95-LT-HT-DET-1-9-M01-PS0304-001-01-15-10',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M01-PS0304-001-01-15-10',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/ca6ccd63f270d603ffa9bdf5a7ced184d63397

In [8]:
experiment.launch_experiment()

2017-06-18 11:36:59,106 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M00-ChvUtd-000-01-15-10
2017-06-18 11:36:59,501 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M01-PS0304-001-01-15-10
2017-06-18 11:36:59,814 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M02-PP0214-000-01-15-10
2017-06-18 11:37:00,076 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M03-ER0429-001-01-15-10
2017-06-18 11:37:00,332 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M04-ChvUtd-000-01-15-15
2017-06-18 11:37:00,600 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M05-PS0304-001-01-15-15
2017-06-18 11:37:00,871 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M06-PP0214-000-01-15-15
2017-06-18 11:37:01,131 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M07-ER0429-001-01-15-15
2017-06-18 11:37:01,391 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-9-M08-ChvUtd-000-01-20-10
2

{'experiment_id': 'PTA-95-LT-HT-DET-1-9',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-9/deps/M00-ChvUtd-000-01-15-10.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-9',
   'job_id': 'PTA-95-LT-HT-DET-1-9-M00-ChvUtd-000-01-15-10',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Sun, 18 Jun 2017 16:34:45 GMT',
      'via': '1.1 67c32d6a47dbd79e174f0928e7b015f3.cloudfront.net (CloudFront)',
      'x-amz-cf-id': '_Qz9M9W2seuTmigvAjVmgdPrBqtGjkFt1Ql9ttuPKlTKFKXeTk0ifw==',
      'x-amzn-requestid': '095627e3-5444-11e7-9c61-4d57fb7d2d17',
      'x-amzn-trace-id': 'sampled=0;root=1-5946aba5-5d43116c2c67ff81b7f4fec9',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': '095627e3-5444-11e7-9c61-4d57fb7d2d17',


In [9]:
import pandas as pd

In [10]:
data = pd.DataFrame({'mota':[.677,.675,.675,.674,.673,.676,.671,.673,.874,.873,.874,.873,.871,.872,.871,.872,.798,
                             .797,.798,.798,.797,.797,.797,.797,.771,.767,.772,.768,.771,.771,.771,.772],
              'lt':[1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2],
              'ht':[15,15,20,20,15,15,20,20,15,15,20,20,15,15,20,20,15,15,20,20,15,15,20,20,15,15,20,20,15,15,20,20],
              'det':[10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15,10,15],
             'game':['ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','PP0214','PP0214',
                     'PP0214','PP0214','PP0214','PP0214','PP0214','PP0214','ChvUtd','ChvUtd','ChvUtd','ChvUtd',
                     'ChvUtd','ChvUtd','ChvUtd','ChvUtd','PS0304','PS0304','PS0304','PS0304','PS0304','PS0304',
                     'PS0304','PS0304']})

In [12]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [13]:
output_notebook()

Loading BokehJS ...

In [14]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections
ht lt det                                      
15 1  10   0.78000     1         15          10
      15   0.77800     1         15          15
   2  10   0.77800     2         15          10
      15   0.77900     2         15          15
20 1  10   0.77975     1         20          10
      15   0.77825     1         20          15
   2  10   0.77750     2         20          10
      15   0.77850     2         20          15

In [15]:
scat_lt_ht_det_1 = Scatter(data_lt_ht_det, x='lost', y='mota', color='detections', marker='hungarian')
show(scat_lt_ht_det_1)